<a href="https://colab.research.google.com/github/Aditi0712/PDFchat_langchain/blob/main/Copy_of_geekrabittask_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Insurance Question Answering System Using LangChain and Open Source LLMs

Created By: Aditi Sharma






Step 1: Installing the required Libraries

In [1]:
!pip install -q accelerate einops bitsandbytes kaleido openai python-multipart tiktoken cohere langchain pypdf faiss-cpu torch chromadb transformers sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.3/222.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.9/283.9 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -q --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.3 MB/s eta 0:00:00


Step 2: Importing the required modules

In [3]:
from langchain_community.document_loaders import PyPDFLoader
import transformers
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma
import torch
from langchain import HuggingFaceHub
from langchain.chains.question_answering import load_qa_chain

Step 3: PDF Ingestion using langchain's PyPDFLoader

In [6]:
loader = PyPDFLoader("/content/insurance-industry-in-canada.pdf")
documents=loader.load()

Step 4: Document Preparation by splitting into manageable chunks

In [7]:
text_splitter=CharacterTextSplitter(
    separator='\n',
    chunk_size=1000,
    chunk_overlap=200,
)
texts=text_splitter.split_documents(documents)

Step 4: Creating embeddings

In [13]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

Step 5: Storing the embedding in the vectorstore

In [14]:
vectorstore= Chroma.from_documents(texts,embeddings)

Step 6: Integrating the open source llm

In [15]:
from google.colab import userdata
hg_token=userdata.get('HuggingFace')

In [16]:
model='tiiuae/falcon-7b-instruct'
llm=HuggingFaceHub(huggingfacehub_api_token=hg_token, repo_id=model, model_kwargs={"temperature":0.3, "max_new_token":2000})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


Step 7: Perform similarity search in vectorstore for contextual resemblance

In [17]:
#docs=vectorstore.similarity_search(query, k=7)

 Step 8: Creating a q&a chain using langchain

In [20]:
chain=load_qa_chain(llm=llm, chain_type="stuff")

Testing:

In [22]:
samples = [
    "What are the key regulations affecting insurance companies in Canada?",
    "How has technology impacted the insurance industry?",
    "How many life insurance companies were in 1985 in Canada?",
    "What policy issues are addressed in the section on industrial development, and how do they impact the insurance industry?"
]

for question in samples:
    # Perform similarity search and run the question-answering chain
    docs = vectorstore.similarity_search(question, k=5)
    result = chain.run(input_documents=docs, question=question)
    # Evaluate and document results
    print(f"Question: {question}")
    print(f"Answer: {result}")
    print("\n---\n")


Question: What are the key regulations affecting insurance companies in Canada?
Answer: 
The key regulations affecting insurance companies in Canada include:
- Financial requirements for insurance companies
- Investment restrictions
- Licensing requirements
- Consumer protection
- Financial reporting and auditing
- Insurance company solvency requirements
- Financial assistance and support
- Taxation of insurance companies
- Anti-competitive practices
- Product liability and consumer protection
- Environmental liability
- Health and safety requirements
- Legal requirements for insurance companies
- Information security and privacy
- Data security and privacy
-

---

Question: How has technology impacted the insurance industry?
Answer: 
The insurance industry has been impacted by technology in a number of ways. Technology has enabled the industry to streamline processes, such as underwriting and claims processing, and has allowed for the development of new products and services. Technolo